# Towards Cellular-Level Alignment: A Coarse-to-Fine Approach for Multistain Whole Slide Image Registration

This notebook demonstrates the complete workflow for Whole Slide Image (WSI) registration using rigid and non-rigid techniques with nuclei-based analysis.

## Overview
- **Coarse Registration**: Initial coarse alignment using traditional techniques
- **Fine Shape-aware Nuclei based Registration**: Point cloud-based fine alignment and coherent Point Drift (CPD) for local deformation
- **Interactive Visualization**: Bokeh plots for analysis


## 1. Setup and Imports

In [1]:
import sys
import os

# Add project root to PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # adjust if needed
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [2]:

%matplotlib inline
%load_ext autoreload
%autoreload 2
import SimpleITK as sitk
import numpy as np
from matplotlib import pyplot as plt

from core.utils.imports import *
from core.config import *
from core.preprocessing.padding import *
from core.preprocessing.preprocessing import *
from core.registration.registration import *
from core.evaluation.evaluation import *
from core.visualization.visualization import *
from core.preprocessing.nuclei_analysis import *
from core.preprocessing.stainnorm import *
from core.registration.nonrigid import *
from core.cpd import *


# Setup Bokeh for notebook output
setup_bokeh_notebook()

print("✅ All modules imported successfully!")
print(f"Source WSI: {SOURCE_WSI_PATH}")
print(f"Target WSI: {TARGET_WSI_PATH}")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


|2025-10-27|23:29:52.131| [WARNING] /home/u5552013/miniconda3/envs/tiatoolbox-reg-vis/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()



Loading BokehJS ...

✅ All modules imported successfully!
Source WSI: /home/u5552013/Nextcloud/HYRECO/Data/Image/ki67_533.tif
Target WSI: /home/u5552013/Nextcloud/HYRECO/Data/Image/he_533.tif


## 2. Configuration Check

Verify that all file paths are correct and files exist.

In [ ]:
import os

# Check if files exist
files_to_check = [
    SOURCE_WSI_PATH,
    TARGET_WSI_PATH,
    FIXED_POINTS_PATH,
    MOVING_POINTS_PATH
]

print("File existence check:")
for file_path in files_to_check:
    exists = os.path.exists(file_path)
    status = "✅" if exists else "❌"
    print(f"{status} {file_path}")

# Display current parameters
print("\nCurrent Parameters:")
print(f"- Preprocessing Resolution: {PREPROCESSING_RESOLUTION}")
print(f"- Registration Resolution: {REGISTRATION_RESOLUTION}")
print(f"- Patch Size: {PATCH_SIZE}")
print(f"- Fixed Threshold: {FIXED_THRESHOLD}")
print(f"- Moving Threshold: {MOVING_THRESHOLD}")
print(f"- Min Nuclei Area: {MIN_NUCLEI_AREA}")

## 3. Load and Preprocess WSIs

In [ ]:
# Load WSI images
print("Loading WSI images...")
source_wsi, target_wsi, source, target = load_wsi_images(
    SOURCE_WSI_PATH, TARGET_WSI_PATH, PREPROCESSING_RESOLUTION
)

print(f"\nLoaded images:")
print(f"Source shape: {source.shape}")
print(f"Target shape: {target.shape}")

In [ ]:
# Preprocess images
print("Preprocessing images...")
# source_prep,target_prep=source,target
source_prep, target_prep,padding_params =pad_images(source, target)

# Extract tissue masks
print("Extracting tissue masks...")
source_mask, target_mask = extract_tissue_masks(source_prep, target_prep, artefacts=False)

print("✅ Preprocessing completed!")

# 4. Visualize Image and Tissue Masks

In [ ]:
# Display original images side by side
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].imshow(source_prep)
axes[0, 0].set_title('Source Image (Moving)')
axes[0, 0].axis('off')

axes[0, 1].imshow(target_prep)
axes[0, 1].set_title('Target Image (Fixed)')
axes[0, 1].axis('off')

axes[1, 0].imshow(source_mask, cmap='gray')
axes[1, 0].set_title('Source Tissue Mask')
axes[1, 0].axis('off')

axes[1, 1].imshow(target_mask, cmap='gray')
axes[1, 1].set_title('Target Tissue Mask')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 5. Coarse Registration

In [ ]:
# Perform rigid registration
print("Performing rigid registration...")
moving_img_transformed, final_transform = perform_rigid_registration(
    source_prep, target_prep, source_mask, target_mask
)
visualize_overlays(target_prep, source_prep, moving_img_transformed)


In [ ]:
u_x, u_y = util.rigid_dot(source_prep,np.linalg.inv(final_transform))
deformation_field = np.stack(( u_x, u_y), axis=-1)
sitk_image = sitk.GetImageFromArray(deformation_field)


def tensor_to_rgb_numpy(tensor):
    # (1, 1, H, W) -> (3, H, W) -> (H, W, 3)
    tensor_rgb = tensor.squeeze().repeat(3, 1, 1)
    return tensor_rgb.permute(1, 2, 0).detach().cpu().numpy()

# coarse nonrigid

displacement_field,warped_source= elastic_image_registration(
   moving_img_transformed,
   target_prep,
     compute_device='cuda'  
)
print("non rigid displacement field",displacement_field.shape)

visualize_overlays(target_prep, source_prep,  tensor_to_rgb_numpy(warped_source))




In [ ]:
disp_field_np=util.tc_df_to_np_df(displacement_field)
w_x,w_y=util.compose_vector_fields(u_x, u_y, disp_field_np[0], disp_field_np[1])
deformation_field = np.stack(( w_x, w_y), axis=-1)
sitk_image = sitk.GetImageFromArray(deformation_field)
sitk.WriteImage(sitk_image, './533.mha')

## 6. TIAViz Registration Visualization 

In [ ]:
%%bash 
export BOKEH_ALLOW_WS_ORIGIN=localhost:5007
tiatoolbox visualize --slides "path-to-slides" --overlays "path-to-overlays"


## 7. Patch Visualization

In [ ]:
# Scale transformation for high resolution analysis
transform_40x = scale_transformation_matrix(
    final_transform, PREPROCESSING_RESOLUTION, REGISTRATION_RESOLUTION
)

# Extract patches from target WSI
print("\nExtracting patches...")
fixed_patch_extractor = extract_patches_from_wsi(
    target_wsi, target_mask, PATCH_SIZE, PATCH_STRIDE
)

print(f"Total patches extracted: {len(fixed_patch_extractor)}")

## Coarse  Visualization

In [ ]:
# Select a patch for visualization
patch_idx = 70  # You can change this index
loc = fixed_patch_extractor.coordinate_list[patch_idx]
location = (loc[0], loc[1])

print(f"Visualizing patch {patch_idx} at location {location}")

# Extract regions for comparison
fixed_tile = target_wsi.read_rect(location, VISUALIZATION_SIZE, resolution=40, units="power")
moving_tile = source_wsi.read_rect(location, VISUALIZATION_SIZE, resolution=40, units="power")

# Create transformer and extract transformed tile
tfm = AffineWSITransformer(source_wsi, transform_40x)
transformed_tile = tfm.read_rect(location=location, size=VISUALIZATION_SIZE, resolution=0, units="level")

# Visualize patches
visualize_patches(fixed_tile, moving_tile, transformed_tile)

In [ ]:
visualize_overlays(fixed_tile, moving_tile, transformed_tile)

## 8. Interactive Nuclei Visualization

In [ ]:

FIXED_NUCLEI_CSV="/home/u5552013/Nextcloud/HYRECO/Data/nuclei_points/he_533_nuclei.csv"
MOVING_NUCLEI_CSV="/home/u5552013/Nextcloud/HYRECO/Data/nuclei_points/ki67_533_nuclei.csv"

# Load nuclei coordinates
moving_df = load_nuclei_coordinates(MOVING_NUCLEI_CSV)
fixed_df = load_nuclei_coordinates(FIXED_NUCLEI_CSV)

print(f"Loaded nuclei data:")
print(f"- Fixed nuclei: {len(fixed_df)}")
print(f"- Moving nuclei: {len(moving_df)}")

# Create basic nuclei overlay plot
print("\nCreating interactive nuclei overlay plot...")
plot1 = create_nuclei_overlay_plot(moving_df, fixed_df, 
                                  "Original Nuclei Coordinates (Before Registration)")
show_plot(plot1)

In [ ]:
deformation_field, moving_updated, fixed_points, moving_points= compute_deformation_and_apply(    source_prep,
    final_transform,
    displacement_field,
    moving_df,
    fixed_df,
    padding_params,
    util,
    pad_landmarks)

In [ ]:
visualize_cluster_alignment(
    fixed_points,
    moving_points,
    moving_updated,
    # fixed_df=None,
    # moving_df=None,
    figsize=(10, 10),
    title='Cluster Centers: Fixed, Original Moving, and Transformed',
    save_path=None
)

## 10. Shape-Aware Point Set Registration

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree


class ShapeAwarePointSetRegistration:
    def __init__(self, fixed_points, moving_points,
                 shape_attribute=None, shape_weight=0.0,
                 max_iterations=50, tolerance=1e-6,
                 allow_scaling=False):

        # --- Convert inputs to numpy arrays ---
        if isinstance(fixed_points, pd.DataFrame):
            self.fixed = fixed_points[['x', 'y']].values.astype(np.float64)
            self.fixed_shape = (fixed_points[shape_attribute].values.astype(np.float64) 
                               if shape_attribute and shape_attribute in fixed_points else None)
        else:
            self.fixed = np.asarray(fixed_points, dtype=np.float64)
            self.fixed_shape = None

        if isinstance(moving_points, pd.DataFrame):
            self.moving = moving_points[['x', 'y']].values.astype(np.float64)
            self.moving_shape = (moving_points[shape_attribute].values.astype(np.float64)
                                if shape_attribute and shape_attribute in moving_points else None)
        else:
            self.moving = np.asarray(moving_points, dtype=np.float64)
            self.moving_shape = None

        if self.fixed.shape[1] != 2 or self.moving.shape[1] != 2:
            raise ValueError("Both fixed_points and moving_points must have exactly 2 columns (x, y).")

        self.shape_weight = np.clip(shape_weight, 0.0, 1.0)
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.allow_scaling = allow_scaling

        # Normalize shape attributes for better weighting
        if self.fixed_shape is not None and self.moving_shape is not None:
            shape_std = np.std(self.fixed_shape)
            if shape_std > 1e-9:
                self.shape_scale = shape_std
            else:
                self.shape_scale = 1.0
        else:
            self.shape_scale = 1.0

        self.rotation = 0.0
        self.scale = 1.0
        self.translation = np.zeros(2)

    def _apply_transform(self, points, R, t, s):
        """Apply similarity transform with vectorized operations."""
        return s * np.dot(points, R.T) + t

    def _estimate_rigid_transform(self, A, B, weights=None):
        """Estimate weighted similarity transform (rotation, translation, scale)."""
        if weights is None:
            weights = np.ones(len(A))
        
        weights = weights / np.sum(weights)  # normalize weights
        
        # Weighted centroids
        centroid_A = np.sum(A * weights[:, np.newaxis], axis=0)
        centroid_B = np.sum(B * weights[:, np.newaxis], axis=0)
        
        AA = A - centroid_A
        BB = B - centroid_B
        
        # Weighted covariance
        H = np.dot(AA.T, BB * weights[:, np.newaxis])
        
        U, _, Vt = np.linalg.svd(H)
        R = np.dot(Vt.T, U.T)
        
        # Ensure proper rotation (det = 1)
        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = np.dot(Vt.T, U.T)

        if self.allow_scaling:
            num = np.sum(weights[:, np.newaxis] * BB * np.dot(AA, R))
            den = np.sum(weights[:, np.newaxis] * AA * AA)
            s = num / (den + 1e-9)
            s = max(0.1, min(s, 10.0))  # clamp scale to reasonable range
        else:
            s = 1.0

        t = centroid_B - s * np.dot(R, centroid_A)
        return R, t, s

    def register(self):
        """Perform iterative closest point registration with shape awareness."""
        fixed_xy = self.fixed
        moving_xy = self.moving
        n_moving = len(moving_xy)

        R = np.eye(2)
        t = np.zeros(2)
        s = 1.0
        prev_error = np.inf

        # Pre-compute spatial distance scale for normalization
        spatial_std = np.std(fixed_xy, axis=0).mean()
        if spatial_std < 1e-9:
            spatial_std = 1.0

        for it in range(self.max_iterations):
            transformed = self._apply_transform(moving_xy, R, t, s)

            # --- Step 1: Find spatial nearest neighbors ---
            kdtree = KDTree(fixed_xy)
            spatial_dists, spatial_idx = kdtree.query(transformed, k=1)

            # --- Step 2: Refine correspondences with shape information ---
            if self.shape_weight > 0 and self.fixed_shape is not None and self.moving_shape is not None:
                # Normalize spatial distances
                norm_spatial_dists = spatial_dists / spatial_std
                
                # Compute shape distances
                shape_dists = np.abs(self.fixed_shape[spatial_idx] - self.moving_shape) / self.shape_scale
                
                # Combined distance metric
                combined_dists = (1 - self.shape_weight) * norm_spatial_dists + self.shape_weight * shape_dists
                
                # For better matching, consider k-nearest neighbors and pick best combined match
                k_neighbors = min(5, len(fixed_xy))
                nn_dists, nn_idx = kdtree.query(transformed, k=k_neighbors)
                
                best_idx = np.zeros(n_moving, dtype=int)
                for i in range(n_moving):
                    candidates = nn_idx[i]
                    spatial_d = nn_dists[i] / spatial_std
                    shape_d = np.abs(self.fixed_shape[candidates] - self.moving_shape[i]) / self.shape_scale
                    combined = (1 - self.shape_weight) * spatial_d + self.shape_weight * shape_d
                    best_idx[i] = candidates[np.argmin(combined)]
                
                idx = best_idx
                matched_fixed = fixed_xy[idx]
                
                # Weight matches by inverse distance for robustness
                weights = 1.0 / (combined_dists + 1e-6)
            else:
                idx = spatial_idx
                matched_fixed = fixed_xy[idx]
                weights = 1.0 / (spatial_dists + 1e-6)

            # --- Step 3: Estimate transform with weighted least squares ---
            R_new, t_new, s_new = self._estimate_rigid_transform(moving_xy, matched_fixed, weights)

            # --- Step 4: Update transform ---
            R = R_new
            t = t_new
            s = s_new

            # --- Step 5: Compute alignment error ---
            transformed_new = self._apply_transform(moving_xy, R, t, s)
            errors = np.linalg.norm(matched_fixed - transformed_new, axis=1)
            error = np.mean(errors)
            
            # Check convergence
            if it > 0 and np.abs(prev_error - error) < self.tolerance * max(prev_error, 1.0):
                break
            prev_error = error

        # Final transformation
        transformed_final = self._apply_transform(moving_xy, R, t, s)

        self.rotation = np.arctan2(R[1, 0], R[0, 0])
        self.translation = t
        self.scale = s
        self.final_error = prev_error
        self.num_iterations = it + 1

        self.registered_points = pd.DataFrame({
            'x': moving_xy[:, 0],
            'y': moving_xy[:, 1],
            'registered_x': transformed_final[:, 0],
            'registered_y': transformed_final[:, 1]
        })

        return self.registered_points

    def get_transformation_matrix(self):
        """Return 3x3 homogeneous transformation matrix."""
        theta = self.rotation
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        tx, ty = self.translation
        s = self.scale
        return np.array([
            [s * cos_t, -s * sin_t, tx],
            [s * sin_t,  s * cos_t, ty],
            [0,          0,         1]
        ])


def perform_shape_aware_registration(fixed_df, moving_df,
                                     shape_attribute=None,
                                     shape_weight=0.0,
                                     max_iterations=50,
                                     tolerance=1e-6,
                                     allow_scaling=False):
    """
    Convenience function for shape-aware point set registration.
    
    Returns:
        registrator: The registration object with transformation parameters
        transform_matrix: 3x3 homogeneous transformation matrix
        coords: Nx2 array of registered coordinates
    """
    registrator = ShapeAwarePointSetRegistration(
        fixed_df, moving_df,
        shape_attribute=shape_attribute,
        shape_weight=shape_weight,
        max_iterations=max_iterations,
        tolerance=tolerance,
        allow_scaling=allow_scaling
    )
    registered_points = registrator.register()
    transform_matrix = registrator.get_transformation_matrix()
    coords = registered_points[['registered_x', 'registered_y']].values
    return registrator, transform_matrix, coords

In [ ]:
# Perform shape-aware registration 
print("Performing Shape-Aware Point Set Registration...")

#  No of subsampling points can be adjusted
fixed_subsample =util.skip_subsample(fixed_df, n_samples=1000)
moving_subsample = util.skip_subsample(moving_df, n_samples=1000)

#  fine pointset registration
shape_registrator,shape_transform, shape_transformed_coords   = perform_shape_aware_registration(
    fixed_points,moving_updated,
    shape_attribute=None,
    shape_weight=0.3,  # 30% weight for shape, 70% for spatial distance
    max_iterations=100,
    tolerance=1e-11
)


In [ ]:
# # #  icp registration
# transformation, transformed_points=perform_icp_registration( moving_updated,fixed_points,threshold=5000000)
# T_sub = np.delete(np.delete(transformation, 2, axis=0), 2, axis=1)


In [ ]:
fixed_subsample =util.skip_subsample(fixed_points, n_samples=1000)
moving_subsample = util.skip_subsample(shape_transformed_coords, n_samples=1000)

In [ ]:
# X = fixed_subsample[['global_x', 'global_y']].values
# Y = moving_subsample[['global_x','global_y']].values
X=fixed_subsample
Y=moving_subsample
print('Non-rigid CPD:')
cpd_nonrigid = CPD(method='nonrigid')
nonrigid_transformed_coords , P = cpd_nonrigid(X, Y, w=0, max_iterations=50)
# print(f'Non-rigid alignment: {np.argmax(P, axis=1)}\n')


In [ ]:


# from scipy.interpolate import griddata

# # Existing grid
# H, W = u_x.shape
# grid_y, grid_x = np.mgrid[0:H, 0:W]

# # Sparse CPD displacement
# cpd_points = X  # (N,2)
# dx = disp_field_np[:,0]
# dy = disp_field_np[:,1]

# # Interpolate to full grid
# full_dx = griddata(cpd_points, dx, (grid_x, grid_y), method='cubic', fill_value=0)
# full_dy = griddata(cpd_points, dy, (grid_x, grid_y), method='cubic', fill_value=0)

# # Now compose
# w_x, w_y = util.compose_vector_fields(u_x, u_y, full_dx, full_dy)
# deformation_field = np.stack((w_x, w_y), axis=-1)

# # Save
# sitk_image = sitk.GetImageFromArray(deformation_field)
# sitk.WriteImage(sitk_image, './cpd_deformation_field.mha')



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycpd import DeformableRegistration as CPD

# Assume X (fixed) and Y (moving) are numpy arrays of shape (N, 2) or (N, 3)
# Y = moving_subsample
X=fixed_subsample
Y=moving_subsample
print('Non-rigid CPD:')
cpd_nonrigid = CPD(X=X, Y=Y, w=0.05,beta=1.5, lambda_=2.0, max_iterations=200)
nonrigid_transformed_coords, P = cpd_nonrigid.register()

# --- Visualization ---
fig = plt.figure(figsize=(12, 5))

# Before registration
ax1 = fig.add_subplot(1, 2, 1)
ax1.scatter(X[:, 0], X[:, 1], c='red', label='Fixed (X)', alpha=0.7)
ax1.scatter(Y[:, 0], Y[:, 1], c='blue', label='Moving (Y)', alpha=0.7)
ax1.set_title('Before Non-Rigid Registration')
ax1.legend()
ax1.axis('equal')

# After registration
ax2 = fig.add_subplot(1, 2, 2)
ax2.scatter(X[:, 0], X[:, 1], c='red', label='Fixed (X)', alpha=0.7)
ax2.scatter(nonrigid_transformed_coords[:, 0], nonrigid_transformed_coords[:, 1], 
            c='green', label='Transformed (Y)', alpha=0.7)
ax2.set_title('After Non-Rigid Registration')
ax2.legend()
ax2.axis('equal')

plt.tight_layout()
plt.show()


In [ ]:
fine_deform=util.create_deformation_field(shape_transform, source_prep, u_x, u_y, util, output_path='./533_finerigid.mha')

In [ ]:
from tiatoolbox.wsicore.wsireader import TransformedWSIReader
from pathlib import Path
# Select a patch for visualization
patch_idx = 70  # You can change this index
loc = fixed_patch_extractor.coordinate_list[patch_idx]
location = (loc[0], loc[1])

print(f"Visualizing patch {patch_idx} at location {location}")

# Extract regions for comparison
fixed_tile = target_wsi.read_rect(location, size=(5000, 5000), resolution=40, units="power")
moving_tile = source_wsi.read_rect(location, size=(5000, 5000), resolution=40, units="power")

transformed_wsi = TransformedWSIReader(input_img=SOURCE_WSI_PATH, target_img=TARGET_WSI_PATH,  transform=Path('./533_finerigid.mha'))
transformed_tile = transformed_wsi.read_rect( location,   size= (5000, 5000),  resolution=40,    units="power" )

# Visualize patches
visualize_patches(fixed_tile, moving_tile, transformed_tile)

In [ ]:
visualize_overlays(fixed_tile, moving_tile, transformed_tile)

In [ ]:
from scipy.ndimage import map_coordinates
import numpy as np
import pandas as pd

# Create displacement field for spatial transformation analysis
print("Creating displacement field...")

# Scale down coordinates for field generation
scale_factor = 64
source_points_scaled = moving_subsample  / scale_factor
target_points_scaled =nonrigid_transformed_coords / scale_factor



from scipy.interpolate import griddata

# Existing grid
H, W = u_x.shape
grid_y, grid_x = np.mgrid[0:H, 0:W]
displacement_field = create_displacement_field(
    source_points_scaled, target_points_scaled,
    target_prep.shape,
    method=RegistrationParams.INTERPOLATION_METHOD,
    sigma=RegistrationParams.DISPLACEMENT_SIGMA,
    max_displacement=RegistrationParams.MAX_DISPLACEMENT
)
fr_x,fr_y=util.compose_vector_fields( w_x, w_y, displacement_field[..., 0],displacement_field[..., 1])

deformation_field = np.stack(( w_x, w_y), axis=-1)
sitk_image = sitk.GetImageFromArray(deformation_field)
sitk.WriteImage(sitk_image, './533_nonrigid.mha')


In [ ]:
from tiatoolbox.wsicore.wsireader import TransformedWSIReader
from pathlib import Path
# Select a patch for visualization
patch_idx = 70  # You can change this index
loc = fixed_patch_extractor.coordinate_list[patch_idx]
location = (loc[0], loc[1])

print(f"Visualizing patch {patch_idx} at location {location}")

# Extract regions for comparison
fixed_tile = target_wsi.read_rect(location, size=(5000, 5000), resolution=40, units="power")
moving_tile = source_wsi.read_rect(location, size=(5000, 5000), resolution=40, units="power")

transformed_wsi = TransformedWSIReader(input_img=SOURCE_WSI_PATH, target_img=TARGET_WSI_PATH,  transform=Path('./533_nonrigid.mha'))
transformed_tile = transformed_wsi.read_rect( location,   size= (5000, 5000),  resolution=40,    units="power" )

# Visualize patches
visualize_patches(fixed_tile, moving_tile, transformed_tile)

In [ ]:
visualize_overlays(fixed_tile,moving_tile, transformed_tile)

In [ ]:
from tiatoolbox.wsicore.wsireader import TransformedWSIReader
from pathlib import Path
# Select a patch for visualization
patch_idx = 70  # You can change this index
loc = fixed_patch_extractor.coordinate_list[patch_idx]
location = (loc[0], loc[1])

print(f"Visualizing patch {patch_idx} at location {location}")

# Extract regions for comparison
fixed_tile = target_wsi.read_rect(location, size=(5000, 5000), resolution=40, units="power")
moving_tile = source_wsi.read_rect(location, size=(5000, 5000), resolution=40, units="power")

transformed_wsi = TransformedWSIReader(input_img=SOURCE_WSI_PATH, target_img=TARGET_WSI_PATH,  transform=Path('./533_nonrigid.mha'))
transformed_tile = transformed_wsi.read_rect( location,   size= (5000, 5000),  resolution=40,    units="power" )

# Visualize patches
visualize_patches(fixed_tile, moving_tile, transformed_tile)

In [ ]:
visualize_overlays(fixed_tile,moving_tile, transformed_tile)

## 11. TIAViz Registration Visualization

In [ ]:
%%bash 
export BOKEH_ALLOW_WS_ORIGIN=localhost:5007

tiatoolbox visualize --slides "path-to-slides" --overlays "path-to-overlays"
